In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.stats import norm
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

import time
from scipy.ndimage.interpolation import shift
import scipy.stats
import FinanceDataReader as fdr
import yfinance as yf
import seaborn as sns

ModuleNotFoundError: ignored

In [ ]:
def BSprice(PutCall, x, T, K, r, q, sigma):
    d1 =(1/(sigma*np.sqrt(T)))*(np.log(x/K) + (r - q + 0.5*sigma**2)*T)
    d2 = (1/(sigma*np.sqrt(T)))*(np.log(x/K) + (r - q - 0.5*sigma**2)*T)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    Nd1m = norm.cdf(-d1)
    Nd2m = norm.cdf(-d2)

    if PutCall == 'C':
        price1 = x*np.exp(-q*T)*Nd1 - K*np.exp(-r*T)*Nd2
        price2 = np.exp(-q*T)*Nd1
    elif PutCall =='P':
        price1 = K*np.exp(-r*T)*Nd2m - x*np.exp(-q*T)*Nd1m
        price2 = -np.exp(-q*T)*Nd1m

    return(price1,price2)           # returns array


# Hedging Cost 비교

In [ ]:
def hedging_Call(s0,M,mu,K,r,sigma,T,q,PutCall,simulation_number):

    M_s=20/M
    M_s=np.int(M_s)
    dt=M/52
    tt=np.repeat(T,M_s)
    for i in range(M_s-1):
        tt[i+1]=tt[i]-dt
    tt=np.append(tt,0)



    # Generating random
    generating_number=int(round(T*(1/dt)))+1

    # generating stock path
    z=np.random.randn(simulation_number,generating_number)
    stock_path_bm=np.exp((mu-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*z)
    stock_path_bm[:,0]=1
    stock_price=stock_path_bm.cumprod(1)*s0

    # Calculating Delta
    Delta=BSprice(PutCall,stock_price, tt, K, r, q, sigma)[1]
    Delta_diff=Delta[:,1:]-Delta[:,:-1]
    Delta_diff.shape

    initial_array=np.repeat(Delta[0][0],Delta_diff.shape[0]).reshape(Delta_diff.shape[0],1)
    Delta_change=np.concatenate([initial_array,Delta_diff],axis=1)

    # Delta 변화에따른 stock trade
    shares_purchased=Delta_change*100000
    shares_purchased=shares_purchased.round()  #정수만 purchase



    cost_purchased=shares_purchased*stock_price
    #trans_cost=np.where(cost_purchased<0,cost_purchased*0.0002,0) #transaction Cost
    trans_cost=cost_purchased*0.00015
    trans_cost=abs(trans_cost)
    transaction=np.cumsum(trans_cost,axis=1)
    cum_cost=np.cumsum(cost_purchased,axis=1)

    # Interest Cost 계산
    for i in range(1,cum_cost.shape[1]):

        cum_cost[:,i]=cum_cost[:,i-1]*np.exp(r*M/52)+cost_purchased[:,i]


    # Final cost 계산
    Final_cost=cum_cost[:,-1]
    hedge_cost=np.where(stock_price[:,-1]>50,Final_cost-5000000,Final_cost)

    Final_cost_tr=cum_cost[:,-1]+transaction[:,-1]
    hedge_cost_tr=np.where(stock_price[:,-1]>50,Final_cost_tr-5000000,Final_cost_tr)

    Mean=np.mean(hedge_cost)
    Performance_MSR=np.std(hedge_cost)/240000


    Mean_tr=np.mean(hedge_cost_tr)
    Performance_MSR_tr=np.std(hedge_cost_tr)/240000

    return(Mean,Performance_MSR,Mean_tr,Performance_MSR_tr)



In [ ]:
M=[5,4,2,1,0.5,0.25]
simulation_number=1000
s0=49
K=50
mu=0.13
r=0.05
sigma=0.2
q=0
PutCall='C'
T=20/52

In [ ]:
for i in M:

    Mean,MSR,Mean_tr,MSR_tr=hedging_Call(s0,i,mu,K,r,sigma,T,q,PutCall,simulation_number)

    if i==5:
        m5=Mean
        m5_tr=Mean_tr
        std5=MSR
        std5_tr=MSR_tr


    if i==4:
        m4=Mean
        m4_tr=Mean_tr
        std4=MSR
        std4_tr=MSR_tr

    if i==2:
        m2=Mean
        m2_tr=Mean_tr
        std2=MSR
        std2_tr=MSR_tr


    if i==1:
        m1=Mean
        m1_tr=Mean_tr
        std1=MSR
        std1_tr=MSR_tr

    if i==0.5:
        m05=Mean
        m05_tr=Mean_tr
        std05=MSR
        std05_tr=MSR_tr

    if i==0.25:

        m025=Mean
        m025_tr=Mean_tr
        std025=MSR
        std025_tr=MSR_tr



### without transaction cost

In [ ]:
df=pd.DataFrame([[m5/100000,m4/100000,m2/100000,m1/100000,m05/100000,m025/100000],[std5,std4,std2,std1,std05,std025]],columns=[5,4,2,1,0.5,0.25])
df.index=['Mean',"Performance MSR"]
df.columns.names=['Rebalance']
df

### transaction cost included

In [ ]:
df=pd.DataFrame([[m5_tr/100000,m4_tr/100000,m2_tr/100000,m1_tr/100000,m05_tr/100000,m025_tr/100000],[std5_tr,std4_tr,std2_tr,std1_tr,std05_tr,std025_tr]],columns=[5,4,2,1,0.5,0.25])
df.index=['Mean',"Performance MSR"]
df.columns.names=['Rebalance']
df

In [ ]:

M=1
M_s=20/M
M_s=np.int(M_s)
dt=M/52
tt=np.repeat(T,M_s)
for i in range(M_s-1):
    tt[i+1]=tt[i]-dt
tt=np.append(tt,0)



# Generating random
generating_number=int(round(T*(1/dt)))+1

# generating stock path
z=np.random.randn(simulation_number,generating_number)
stock_path_bm=np.exp((mu-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*z)
stock_path_bm[:,0]=1
stock_price=stock_path_bm.cumprod(1)*s0

In [ ]:
for i in range(1000):
    plt.plot(stock_price[i])

In [ ]:
-----------절취선-----------------    #여기까지 기존 과제

### 현재가 50,000원 strike = 55,000원  수량# 100,000 계약
### OPT 매도 포지션일때 내재 변동성 45%로 마진을 쌓아 놓고 Hedging PNL을 계산한다
### Int cost =4% daily calculation
### Transaction Cost= 0.1% at selling

In [ ]:
PutCall='C'    # put / call 중에 어떤 상품인가
x=50    # 기초자산 현재가격
T=20/52    # 만기
K=55    # strike price
r=0.05   # 자금비용 & risk-free rate
q=0    # 배당 (무시)
sigma=0.46    # impvol



c_price=BSprice(PutCall, x, T, K, r, q, sigma)[0]
c_price*100000   #Margin

In [ ]:
simulation_number=1000
s0=50
K=55
mu=0.13
r=0.04
sigma=0.4
q=0
PutCall='C'
T=20/52

In [ ]:
def hedging_C(s0,mu,K,r,sigma,T,q,PutCall,simulation_number):
    M=1
    M_s=20/M
    M_s=np.int(M_s)
    dt=M/52
    tt=np.repeat(T,M_s)
    for i in range(M_s-1):
        tt[i+1]=tt[i]-dt
    tt=np.append(tt,0)



    # Generating random
    generating_number=int(round(T*(1/dt)))+1

    # generating stock path
    z=np.random.randn(simulation_number,generating_number)
    stock_path_bm=np.exp((mu-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*z)
    stock_path_bm[:,0]=1
    stock_price=stock_path_bm.cumprod(1)*s0

    # Calculating Delta
    Delta=BSprice(PutCall,stock_price, tt, K, r, q, sigma)[1]
    Delta_diff=Delta[:,1:]-Delta[:,:-1]
    Delta_diff.shape

    initial_array=np.repeat(Delta[0][0],Delta_diff.shape[0]).reshape(Delta_diff.shape[0],1)
    Delta_change=np.concatenate([initial_array,Delta_diff],axis=1)

    # Delta 변화에따른 stock trade
    shares_purchased=Delta_change*100000
    shares_purchased=shares_purchased.round()  #정수만 purchase



    cost_purchased=shares_purchased*stock_price
    trans_cost=np.where(cost_purchased<0,cost_purchased*0.001,0) #transaction Cost # 매도시에만 10BP
    #trans_cost=cost_purchased*0.00015
    trans_cost=abs(trans_cost)
    transaction=np.cumsum(trans_cost,axis=1)
    cum_cost=np.cumsum(cost_purchased,axis=1)

    # Interest Cost 계산
    for i in range(1,cum_cost.shape[1]):

        cum_cost[:,i]=cum_cost[:,i-1]*np.exp(r*M/52)+cost_purchased[:,i]


    # Final cost 계산
    Final_cost=cum_cost[:,-1]
    hedge_cost=np.where(stock_price[:,-1]>K,Final_cost-K*100000,Final_cost)

    Final_cost_tr=cum_cost[:,-1]+transaction[:,-1]
    hedge_cost_tr=np.where(stock_price[:,-1]>K,Final_cost_tr-K*100000,Final_cost_tr)

    ATM_C=np.where((stock_price[:,-1]<1.01*K)&(stock_price[:,-1]>0.99*K),hedge_cost,0)    #ATM 으로 끝났을때
    ITM_C=np.where(stock_price[:,-1]>K,hedge_cost,0)    #ITM 으로 끝났을때
    OTM_C=np.where(stock_price[:,-1]<K,hedge_cost,0)    #OTM 으로 끝났을때

    ATM_Case=ATM_C[ATM_C!=0]
    ITM_Case=ITM_C[ITM_C!=0]
    OTM_Case=OTM_C[OTM_C!=0]


    hedging_PNL=(c_price*100000-hedge_cost)

    Mean=np.mean(hedge_cost)    # transcation cost 포함하지 않은 hedging cost
    Performance_MSR=np.std(hedge_cost)/240000


    Mean_tr=np.mean(hedge_cost_tr)   #transaction cost 포함함 hedging cost
    Performance_MSR_tr=np.std(hedge_cost_tr)/240000
    Mean_hedging_PNL=np.mean(hedging_PNL)


    return(Mean,Performance_MSR,Mean_tr,Performance_MSR_tr,np.mean(ITM_Case),np.mean(OTM_Case),np.mean(ATM_Case),Mean_hedging_PNL)




In [ ]:
s0=49
K=50
mu=0.13
r=0.05
sigma=0.2
q=0
PutCall='C'
T=20/52
simulation_number=1000


## transaction cost 비교

In [ ]:
Mean_without_transcost=np.zeros(1000)
Mean_with_transcost=np.zeros(1000)
for i in tqdm(range(1000)):

    Mean_without_transcost[i]=hedging_C(s0,mu,K,r,sigma,T,q,PutCall,simulation_number)[0]
    Mean_with_transcost[i]=hedging_C(s0,mu,K,r,sigma,T,q,PutCall,simulation_number)[2]


In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(Mean_without_transcost,color='blue',label='Without Cost')
plt.axvline(x=Mean_without_transcost.mean(),color='black',ls='--',lw=4)

sns.distplot(Mean_with_transcost,color='red',label="Wiht Cost")
plt.axvline(x=Mean_with_transcost.mean(),color='black',ls='--',lw=4)

plt.xlabel("Cost",size=20)
plt.legend()
plt.show()

## Hedging PNL

In [ ]:
a=np.zeros(1000)
for i in tqdm(range(1000)):
    a[i]=hedging_C(s0,mu,K,r,sigma,T,q,PutCall,simulation_number)[7]

In [ ]:
sns.distplot(a,color='blue')
plt.axvline(x=a.mean(),color='black',ls='--',lw=4)

### ITM ATM OTM case

In [ ]:
ITM_=np.zeros(1000)
ATM_=np.zeros(1000)
OTM_=np.zeros(1000)
for i in tqdm(range(1000)):

    ITM_[i]=hedging_C(s0,mu,K,r,sigma,T,q,PutCall,simulation_number)[4]
    OTM_[i]=hedging_C(s0,mu,K,r,sigma,T,q,PutCall,simulation_number)[5]
    ATM_[i]=hedging_C(s0,mu,K,r,sigma,T,q,PutCall,simulation_number)[6]

In [ ]:
plt.figure(figsize=(10,10))
sns.distplot(ITM_,color='blue',label='ITM')
plt.axvline(x=ITM_.mean(),color='black',ls='--',lw=4)

sns.distplot(OTM_,color='red',label="OTM")
plt.axvline(x=OTM_.mean(),color='black',ls='--',lw=4)

sns.distplot(ATM_,color='gray',label="ATM")
plt.axvline(x=ATM_.mean(),color='black',ls='--',lw=4)

plt.xlabel("Cost",size=20)
plt.legend()
plt.show()

In [ ]:
-------------1차 절취선---------------

In [ ]:
kospi_c_all=pd.read_csv("kospi_opt_M11_K355.csv")
kospi_c_all.index=kospi_c_all['일자']
kospi_c_all=kospi_c_all.iloc[:,:11]
kospi_opt_df=kospi_c_all[kospi_c_all['거래량']>50]  # 거래량 50 이상
kospi_opt_df.iloc[-1]

In [ ]:
df_etf=fdr.DataReader('069500')
df_etf=df_etf['Close']


In [ ]:
df_etf_ret=df_etf.pct_change(1).dropna()

In [ ]:
rolling_90_std=df_etf_ret.rolling(90).std()*np.sqrt(252)
rolling_90_std=rolling_90_std[(rolling_90_std.index<='2022-11-10')&(rolling_90_std.index>='2022-07-07')]
rolling_90_std

In [ ]:
df_etf_kospi=df_etf[(df_etf.index>="2022-07-07")&(df_etf.index<="2022-11-10")]
df_etf_kospi

In [ ]:
kospi_200_s=pd.read_csv('Kospi_200_index.csv')
kospi_200_s.index=kospi_200_s['일자']
kospi_200_s=kospi_200_s[(kospi_200_s.index>="2022-07-07")&(kospi_200_s.index<="2022-11-10")]
kospi_200_s=kospi_200_s['종가'].loc[::-1]
kospi_200_s

In [ ]:
def BSprice(PutCall, x, T, K, r, q, sigma):
    d1 =(1/(sigma*np.sqrt(T)))*(np.log(x/K) + (r - q + 0.5*sigma**2)*T)
    d2 = (1/(sigma*np.sqrt(T)))*(np.log(x/K) + (r - q - 0.5*sigma**2)*T)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    Nd1m = norm.cdf(-d1)
    Nd2m = norm.cdf(-d2)

    if PutCall == 'C':
        price1 = x*np.exp(-q*T)*Nd1 - K*np.exp(-r*T)*Nd2
        price2 = np.exp(-q*T)*Nd1
    elif PutCall =='P':
        price1 = K*np.exp(-r*T)*Nd2m - x*np.exp(-q*T)*Nd1m
        price2 = -np.exp(-q*T)*Nd1m

    return(price1,price2)

In [ ]:
PutCall="C"
T=len(kospi_200_s.index)
tt=[]
for i in range(T-1):
    tt.append(T-1-i)
tt.append(0)
tt=np.array(tt)
T=tt
K=355
r=0.0321
sigma=rolling_90_std

In [ ]:
df_etf_kospi*0.01

In [ ]:
BSprice('C',df_etf_kospi*0.01,tt,K,r,0.0179,rolling_90_std)[0]  # Call price

In [ ]:
Delta=BSprice('C',df_etf_kospi*0.01,tt,K,r,0.0179,rolling_90_std)[1]   # Delta
Delta

In [ ]:
Delta_diff=Delta[1:]-Delta[:-1]

In [ ]:
Delta_diff
Delta_change=np.repeat(Delta[0],len(Delta_diff)+1)
Delta_change[1:]=Delta_diff
Delta_change

In [ ]:
Delta_change*100000

In [ ]:
1.39/0.05*250000

In [ ]:
1.4/0.5*250000

In [ ]:
------ 절취선------- 아래는 실패작

In [ ]:
def BSprice_Futures(PutCall, F, T, K, r, sigma):   ### John hull 책 415 페이지 관련내용
    d1 =(1/(sigma*np.sqrt(T)))*(np.log(F/K) + (0.5*sigma**2)*T)
    d2 = (1/(sigma*np.sqrt(T)))*(np.log(F/K) - (0.5*sigma**2)*T)
    Nd1 = norm.cdf(d1)
    Nd2 = norm.cdf(d2)
    Nd1m = norm.cdf(-d1)
    Nd2m = norm.cdf(-d2)

    if PutCall == 'C':
        price1 = F*np.exp(-r*T)*Nd1 - K*np.exp(-r*T)*Nd2
        price2 = np.exp(-r*T)*Nd1
    elif PutCall =='P':
        price1 = K*np.exp(-r*T)*Nd2m - F*np.exp(-r*T)*Nd1m
        price2 = -np.exp(-r*T)*Nd1m

    return(price1,price2)           # returns array


In [ ]:
kospi_c_all=pd.read_csv("kospi_opt_M11_K355.csv")
kospi_c_all.index=kospi_c_all['일자']
kospi_c_all=kospi_c_all.iloc[:,:11]
kospi_opt_df=kospi_c_all[kospi_c_all['거래량']>50]  # 거래량 50 이상
kospi_opt_df.iloc[-1]

In [ ]:
kospi_200=pd.read_csv('Kospi_200_index.csv')
kospi_200.index=kospi_200['일자']
#kospi_200#=#kospi_200[(kospi_200.index>="2022-08-04")&(kospi_200.index<="2022-11-10")]
kospi_200=kospi_200['종가'].loc[::-1]
kospi_200=kospi_200[kospi_200.index=='2022-07-07']
kospi_200


In [ ]:
kospi_200_s=pd.read_csv('Kospi_200_index.csv')
kospi_200_s.index=kospi_200_s['일자']
kospi_200_s=kospi_200_s['종가']
kospi_200_s=kospi_200_s.pct_change(1).dropna()
kospi_200_s=kospi_200_s.loc[::-1]
rolling_90_std=kospi_200_s.rolling(90).std()*np.sqrt(252)
rolling_90_std=rolling_90_std.dropna()
rolling_90_std=rolling_90_std[(rolling_90_std.index<="2022-11-10")&(rolling_90_std.index>="2022-07-07")]
rolling_90_std

In [ ]:
kospi_fut=pd.read_csv("kospi_200_Fut.csv") ## futures
kospi_fut.index=kospi_fut['일자']
kospi_fut=kospi_fut.iloc[:,1:]
kospi_fut=kospi_fut.iloc[:,:10].dropna()
kospi_fut=kospi_fut.loc[::-1]
kospi_fut=kospi_fut[(kospi_fut.index>='2022-07-07')&(kospi_fut.index<='2022-11-10')]
kospi_fut

In [ ]:
# plt.figure(figsize=(20,10))
# kospi_fut=kospi_fut[kospi_fut.index>='2022-07-07']
# plt.plot(kospi_fut['종가'])
# plt.plot(kospi_200[kospi_200.index>='2022-07-07'])


In [ ]:
T=len(kospi_fut.index)
tt=[]
for i in range(T-1):
    tt.append(T-1-i)
tt.append(0)

In [ ]:
PutCall="C"
F=kospi_fut['종가']
T=len(kospi_fut.index)
tt=[]
for i in range(T-1):
    tt.append(T-1-i)
tt.append(0)
tt=np.array(tt)
T=tt
K=355
r=0.0321
sigma=rolling_90_std

In [ ]:
Delta=BSprice_Futures(PutCall, F, T, K, r, sigma)[1]

In [ ]:
import numpy as np

In [ ]:
(np.exp(0.03/12)*(6.23+4.05-2*5.05))/1.25**2

In [ ]:
Delta

In [ ]:
plt.plot(Delta)

In [ ]:
BSprice_Futures(PutCall, F, T, K, r, sigma)[0]

In [ ]:
Delta_diff=Delta[:-1]-Delta[1:]
Delta_change=np.repeat(Delta[0],len(Delta_diff)+1)
Delta_change[1:]=Delta_diff

In [ ]:
Delta_change

In [ ]:
shares_purchased=Delta_change*1000000
shares_purchased=shares_purchased.round()

In [ ]:
np.cumsum(shares_purchased*kospi_fut['종가'])[-1]

In [ ]:
Delta_diff=Delta[:-1]-Delta[1:]


In [ ]:
Delta_diff

In [ ]:
Delta_change=np.repeat(Delta[0],len(Delta_diff)+1)
Delta_change[1:]=Delta_diff

In [ ]:
shares_purchased=Delta_change*1000000
shares_purchased=shares_purchased.round()

In [ ]:
shares_purchased

In [ ]:
np.cumsum(shares_purchased*kospi_fut['종가'])[-1]

In [ ]:
1.39*1000000

In [ ]:
kospi_fut

In [ ]:
T

In [ ]:
kospi_fut['종가']

In [ ]:
Delta_diff.round(3)

In [ ]:
len(Delta_diff)


In [ ]:
fdr.DataReader('069500','2022-01-01')

In [ ]:
8+5+8+8

In [ ]:
9+7+10+10


In [ ]:
--------절취선----------   # 위에까지가 Futures

In [ ]:
s0=50
K=55
mu=0.13
r=0.04
sigma=0.4
q=0
PutCall='C'
T=20/52

In [ ]:

M=1
M_s=20/M
M_s=np.int(M_s)
dt=M/52
tt=np.repeat(T,M_s)
for i in range(M_s-1):
    tt[i+1]=tt[i]-dt
tt=np.append(tt,0)



# Generating random
generating_number=int(round(T*(1/dt)))+1

# generating stock path
z=np.random.randn(simulation_number,generating_number)
stock_path_bm=np.exp((mu-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*z)
stock_path_bm[:,0]=1
stock_price=stock_path_bm.cumprod(1)*s0

# Calculating Delta
Delta=BSprice(PutCall,stock_price, tt, K, r, q, sigma)[1]
Delta_diff=Delta[:,1:]-Delta[:,:-1]
Delta_diff.shape

initial_array=np.repeat(Delta[0][0],Delta_diff.shape[0]).reshape(Delta_diff.shape[0],1)
Delta_change=np.concatenate([initial_array,Delta_diff],axis=1)

# Delta 변화에따른 stock trade
shares_purchased=Delta_change*100000
shares_purchased=shares_purchased.round()  #정수만 purchase



cost_purchased=shares_purchased*stock_price
trans_cost=np.where(cost_purchased<0,cost_purchased*0.001,0) #transaction Cost # 매도시에만 10BP
#trans_cost=cost_purchased*0.00015
trans_cost=abs(trans_cost)
transaction=np.cumsum(trans_cost,axis=1)
cum_cost=np.cumsum(cost_purchased,axis=1)

# Interest Cost 계산
for i in range(1,cum_cost.shape[1]):

    cum_cost[:,i]=cum_cost[:,i-1]*np.exp(r*M/52)+cost_purchased[:,i]


# Final cost 계산
Final_cost=cum_cost[:,-1]
hedge_cost=np.where(stock_price[:,-1]>K,Final_cost-K*100000,Final_cost)

Final_cost_tr=cum_cost[:,-1]+transaction[:,-1]
hedge_cost_tr=np.where(stock_price[:,-1]>K,Final_cost_tr-K*100000,Final_cost_tr)

ATM_C=np.where(stock_price[:,-1]==K,hedge_cost,0)    #ATM 으로 끝났을때
ITM_C=np.where(stock_price[:,-1]>K,hedge_cost,0)    #ITM 으로 끝났을때
OTM_C=np.where(stock_price[:,-1]<K,hedge_cost,0)    #OTM 으로 끝났을때

ATM_Case=ATM_C[ATM_C!=0]
ITM_Case=ITM_C[ITM_C!=0]
OTM_Case=OTM_C[OTM_C!=0]


hedging_PNL=(405567.6897503858-hedge_cost)

Mean=np.mean(hedge_cost)    # transcation cost 포함하지 않은 hedging cost
Performance_MSR=np.std(hedge_cost)/240000


Mean_tr=np.mean(hedge_cost_tr)   #transaction cost 포함함 hedging cost
Performance_MSR_tr=np.std(hedge_cost_tr)/240000
Mean_hedging_PNL=np.mean(hedging_PNL)


Mean,Performance_MSR,Mean_tr,Performance_MSR_tr,np.mean(ITM_Case),np.mean(OTM_Case),np.mean(ATM_Case)





In [ ]:
345*1.05

In [ ]:
yf.download('005930.KS',start='2022-05-13')

In [ ]:
-------- 2차 절취선-----------



In [ ]:
def hedging_Call(s0,M,mu,K,r,sigma,T,q,PutCall,simulation_number):
    M_s=20/M
    M_s=np.int(M_s)
    dt=M/52
    tt=np.repeat(T,M_s)
    for i in range(M_s-1):
        tt[i+1]=tt[i]-dt
    tt=np.append(tt,0)



    # Generating random
    generating_number=int(round(T*(1/dt)))+1

    # generating stock path
    z=np.random.randn(simulation_number,generating_number)
    stock_path_bm=np.exp((mu-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*z)
    stock_path_bm[:,0]=1
    stock_price=stock_path_bm.cumprod(1)*s0

    # Calculating Delta
    Delta=BSprice(PutCall,stock_price, tt, K, r, q, sigma)[1]
    Delta_diff=Delta[:,1:]-Delta[:,:-1]
    Delta_diff.shape

    initial_array=np.repeat(Delta[0][0],Delta_diff.shape[0]).reshape(Delta_diff.shape[0],1)
    Delta_change=np.concatenate([initial_array,Delta_diff],axis=1)

    # Delta 변화에따른 stock trade
    shares_purchased=Delta_change*100000
    shares_purchased=shares_purchased.round()  #정수만 purchase



    cost_purchased=shares_purchased*stock_price
    #trans_cost=np.where(cost_purchased<0,cost_purchased*0.0002,0) #transaction Cost
    trans_cost=cost_purchased*0.00015
    trans_cost=abs(trans_cost)
    transaction=np.cumsum(trans_cost,axis=1)
    cum_cost=np.cumsum(cost_purchased,axis=1)

    # Interest Cost 계산
    for i in range(1,cum_cost.shape[1]):

        cum_cost[:,i]=cum_cost[:,i-1]*np.exp(r*M/52)+cost_purchased[:,i]


    # Final cost 계산
    Final_cost=cum_cost[:,-1]
    hedge_cost=np.where(stock_price[:,-1]>50,Final_cost-5000000,Final_cost)

    Final_cost_tr=cum_cost[:,-1]+transaction[:,-1]
    hedge_cost_tr=np.where(stock_price[:,-1]>50,Final_cost_tr-5000000,Final_cost_tr)

    Mean=np.mean(hedge_cost)
    Performance_MSR=np.std(hedge_cost)/240000


    Mean_tr=np.mean(hedge_cost_tr)
    Performance_MSR_tr=np.std(hedge_cost_tr)/240000

    return(Mean,Performance_MSR,Mean_tr,Performance_MSR_tr)


M=[5,4,2,1,0.5,0.25]
simulation_number=1000
s0=49
K=50
mu=0.13
r=0.05
sigma=0.2
q=0
PutCall='C'
T=20/52
for i in M:

    Mean,MSR,Mean_tr,MSR_tr=hedging_Call(s0,i,mu,K,r,sigma,T,q,PutCall,simulation_number)

    if i==5:
        m5=Mean
        m5_tr=Mean_tr
        std5=MSR
        std5_tr=MSR_tr


    if i==4:
        m4=Mean
        m4_tr=Mean_tr
        std4=MSR
        std4_tr=MSR_tr

    if i==2:
        m2=Mean
        m2_tr=Mean_tr
        std2=MSR
        std2_tr=MSR_tr


    if i==1:
        m1=Mean
        m1_tr=Mean_tr
        std1=MSR
        std1_tr=MSR_tr

    if i==0.5:
        m05=Mean
        m05_tr=Mean_tr
        std05=MSR
        std05_tr=MSR_tr

    if i==0.25:

        m025=Mean
        m025_tr=Mean_tr
        std025=MSR
        std025_tr=MSR_tr


### without transaction cost

In [ ]:
df=pd.DataFrame([[m5_tr/100000,m4_tr/100000,m2_tr/100000,m1_tr/100000,m05_tr/100000,m025_tr/100000],[std5_tr,std4_tr,std2_tr,std1_tr,std05_tr,std025_tr]],columns=[5,4,2,1,0.5,0.25])
df.index=['Mean',"Performance MSR"]
df.columns.names=['Rebalance']
df

In [ ]:
ITM_C=np.where(stock_price[:,-1]>K,hedge_cost,0)
OTM_C=np.where(stock_price[:,-1]<K,hedge_cost,0)
ITM_Case=ITM_C[ITM_C!=0]
OTM_Case=OTM_C[OTM_C!=0]
np.mean(ITM_Case),np.mean(OTM_Case)



In [ ]:
ITM_Case[ITM_Case!=0]


In [ ]:
df_1=pd.DataFrame([stock_price[-1,:],Delta[-1,:],shares_purchased[-1,:],cost_purchased[-1,:],cum_cost[-1,:]],index=['stock','delta','#purch','$purch','cum']).T
df_1.round(3)

In [ ]:
# Generating random
generating_number=int(round(T*(1/dt)))+1

# generating stock path
z=np.random.randn(simulation_number,generating_number)
stock_path_bm=np.exp((mu-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*z)
stock_path_bm[:,0]=1
stock_price=stock_path_bm.cumprod(1)*s0

In [ ]:
stock_price[0]

In [ ]:
PutCall="C"
x=50
T=20/52
K=55
r=0.04
q=0
sigma=0.46

In [ ]:
BSprice(PutCall, x, T, K, r, q, sigma)[0]*100000

In [ ]:
d1 =(1/(sigma*np.sqrt(T)))*(np.log(x/K) + (r - q + 0.5*sigma**2)*T)
d2 = (1/(sigma*np.sqrt(T)))*(np.log(x/K) + (r - q - 0.5*sigma**2)*T)
Nd1 = norm.cdf(d1)
Nd2 = norm.cdf(d2)
Nd1m = norm.cdf(-d1)
Nd2m = norm.cdf(-d2)



# 유의성 검정

In [ ]:
M=[5,4,2,1,0.5,0.25]
simulation_number=1000
s0=49
K=50
mu=0.13
r=0.05
sigma=0.2
q=0
PutCall='C'
T=20/52
Mean_5=[]
std_5=[]
Mean_4=[]
std_4=[]
Mean_2=[]
std_2=[]
Mean_1=[]
std_1=[]
Mean_05=[]
std_05=[]
Mean_025=[]
std_025=[]

Mean_tr_5=[]
std_tr_5=[]
Mean_tr_4=[]
std_tr_4=[]
Mean_tr_2=[]
std_tr_2=[]
Mean_tr_1=[]
std_tr_1=[]
Mean_tr_05=[]
std_tr_05=[]
Mean_tr_025=[]
std_tr_025=[]

for j in tqdm(range(1000)):
    for i in M:

        Mean,MSR,Mean_tr,MSR_tr=hedging_Call(s0,i,mu,K,r,sigma,T,q,PutCall,simulation_number)

        if i==5:
            Mean_5.append(Mean)
            std_5.append(MSR)
            Mean_tr_5.append(Mean_tr)
            std_tr_5.append(MSR_tr)

        if i==4:

            Mean_4.append(Mean)
            std_4.append(MSR)
            Mean_tr_4.append(Mean_tr)
            std_tr_4.append(MSR_tr)

        if i==2:

            Mean_2.append(Mean)
            std_2.append(MSR)
            Mean_tr_2.append(Mean_tr)
            std_tr_2.append(MSR_tr)

        if i==1:

            Mean_1.append(Mean)
            std_1.append(MSR)
            Mean_tr_1.append(Mean_tr)
            std_tr_1.append(MSR_tr)

        if i==0.5:

            Mean_05.append(Mean)
            std_05.append(MSR)
            Mean_tr_05.append(Mean_tr)
            std_tr_05.append(MSR_tr)

        if i==0.25:

            Mean_025.append(Mean)
            std_025.append(MSR)
            Mean_tr_025.append(Mean_tr)
            std_tr_025.append(MSR_tr)



In [ ]:
s0=49
K=50
mu=0.13
r=0.05
sigma=0.2
q=0
PutCall='C'
T=20/52
simulation_number=10000

In [ ]:

M=1
M_s=20/M
M_s=np.int(M_s)
dt=M/52
tt=np.repeat(T,M_s)
for i in range(M_s-1):
    tt[i+1]=tt[i]-dt
tt=np.append(tt,0)



# Generating random
generating_number=int(round(T*(1/dt)))+1

# generating stock path
z=np.random.randn(simulation_number,generating_number)
stock_path_bm=np.exp((mu-0.5*sigma**2)*dt+sigma*np.sqrt(dt)*z)
stock_path_bm[:,0]=1
stock_price=stock_path_bm.cumprod(1)*s0

# Calculating Delta
Delta=BSprice(PutCall,stock_price, tt, K, r, q, sigma)[1]
Delta_diff=Delta[:,1:]-Delta[:,:-1]
Delta_diff.shape

initial_array=np.repeat(Delta[0][0],Delta_diff.shape[0]).reshape(Delta_diff.shape[0],1)
Delta_change=np.concatenate([initial_array,Delta_diff],axis=1)

# Delta 변화에따른 stock trade
shares_purchased=Delta_change*100000
shares_purchased=shares_purchased.round()  #정수만 purchase



cost_purchased=shares_purchased*stock_price
trans_cost=np.where(cost_purchased<0,cost_purchased*0.001,0) #transaction Cost # 매도시에만 10BP
#trans_cost=cost_purchased*0.00015
trans_cost=abs(trans_cost)
transaction=np.cumsum(trans_cost,axis=1)
cum_cost=np.cumsum(cost_purchased,axis=1)

# Interest Cost 계산
for i in range(1,cum_cost.shape[1]):

    cum_cost[:,i]=cum_cost[:,i-1]*np.exp(r*M/52)+cost_purchased[:,i]


# Final cost 계산
Final_cost=cum_cost[:,-1]
hedge_cost=np.where(stock_price[:,-1]>K,Final_cost-K*100000,Final_cost)

Final_cost_tr=cum_cost[:,-1]+transaction[:,-1]
hedge_cost_tr=np.where(stock_price[:,-1]>K,Final_cost_tr-K*100000,Final_cost_tr)

ATM_C=np.where((stock_price[:,-1]<(K+1))&(stock_price[:,-1]>(K-1)),hedge_cost,0)   #ATM 으로 끝났을때
ITM_C=np.where(stock_price[:,-1]>K,hedge_cost,0)    #ITM 으로 끝났을때
OTM_C=np.where(stock_price[:,-1]<K,hedge_cost,0)    #OTM 으로 끝났을때

ATM_Case=ATM_C[ATM_C!=0]
ITM_Case=ITM_C[ITM_C!=0]
OTM_Case=OTM_C[OTM_C!=0]


hedging_PNL=(c_price*100000-hedge_cost)

Mean=np.mean(hedge_cost)    # transcation cost 포함하지 않은 hedging cost
Performance_MSR=np.std(hedge_cost)/240000


Mean_tr=np.mean(hedge_cost_tr)   #transaction cost 포함함 hedging cost
Performance_MSR_tr=np.std(hedge_cost_tr)/240000
Mean_hedging_PNL=np.mean(hedging_PNL)
Mean,Performance_MSR,Mean_tr,Performance_MSR_tr,np.mean(ITM_Case),np.mean(OTM_Case),np.mean(ATM_Case),Mean_hedging_PNL




In [ ]:
ATM_Case